# **Front-running is barely profitable on Moonbeam's StellaSwap**

In [1]:
def quote_with_fees(in_reserve, out_reserve, amount_in=1):
    amount_out = (amount_in * .9975 * out_reserve) / (in_reserve + amount_in * .9975)
    return amount_out

def inverse_quote_with_fees(in_reserve, out_reserve, amount_out):
    amount_in = (400/399) * amount_out * in_reserve / (out_reserve - amount_out)
    return amount_in

# Motivation: A massive StellaSwap transaction
Txn hash: 0x98c8aedcc1b235eff1d8b9e5ad33bbfd3d57357a7f143cf1076262908604c2b1 (WELL -> WGLMR)
```
swapTokensForExactETH(
  amountOut = 10000000000000000000000
  amountInMax = 503282689359348937701448
  path = ['0x511aB53F793683763E5a8829738301368a2411E3', '0xAcc15dC74880C9944775448304B263D191c6077F']
)
```

In [2]:
amountOut = 10000000000000000000000
amountInMax = 503282689359348937701448

# Starting state
reserve0_i = 4040277759327742164205568
reserve1_i = 90922656907358195679232
# Big transaction
amount0_delta = 500527778442395412070400
amount1_delta = -10000000000000000000000
# State after the large txn (slight rounding error)
reserve0_f = 4540805537770137576275968
reserve1_f = 80922656907358199873536

In [3]:
# Sanity checks
assert(quote_with_fees(reserve0_i, reserve1_i, amount0_delta) + amount1_delta == 0)
assert(inverse_quote_with_fees(reserve0_i, reserve1_i, -amount1_delta) - amount0_delta == 0)

print('Before rate:', reserve0_i / reserve1_i) # 44.43642428360197
print('After rate:', reserve0_f / reserve1_f) # 56.112907204326454

# Note that this has slight rounding error
print('Amount deltas in large transaction:', reserve0_f - reserve0_i, reserve1_f - reserve1_i) # 500527778442395412070400 -9999999999999995805696

Before rate: 44.43642428360197
After rate: 56.112907204326454
Amount deltas in large transaction: 500527778442395412070400 -9999999999999995805696


In [4]:
# At the time, each token0 (WELL) was worth roughly .012746 DAI units, which is .012746 * 1e-18 USD and each WGLMR was worth 0.715208 DAI units
token0_to_usd = .012746 * 1e-18
token1_to_usd = 0.715208 * 1e-18

# Front-run strategy: Sandwich trade
1. Swap token0 for token1 before the large transaction
2. Let the large transaction occur, driving the value of token0 up and token1 down
3. Swap token1 for token0 after

In [5]:
# Front-run strategy -> sandwich: Swap token1 for token0 before the giant transaction, then swap token0 for token1 after
r0, r1 = reserve0_i, reserve1_i

# A) Front-run large transaction: swap token0 -> token1
A_amount_token0_in = 1e22
A_amount_token1_out = quote_with_fees(r0, r1, amount_in=A_amount_token0_in)
r0 += A_amount_token0_in
r1 -= A_amount_token1_out

# B) Large transaction occurs
B_amount_token1_out = amountOut
B_amount_token0_in = inverse_quote_with_fees(r0, r1, B_amount_token1_out)
assert(B_amount_token0_in <= amountInMax) # otherwise their transaction will be rejected!
print('3rd party\'s proportion of amountInMax:', B_amount_token0_in / amountInMax)
r0 += B_amount_token0_in
r1 -= B_amount_token1_out

# C) We swap in the reverse direction (token1 -> token0); for simplicity let's say we swap away all the token1's we acquired
C_amount_token1_in = A_amount_token1_out
C_amount_token0_out = quote_with_fees(r1, r0, C_amount_token1_in)

# Net gain:
print('\nResult')
print(f'A) [Our front-run]       token0_in:  {A_amount_token0_in}  (-${A_amount_token0_in * token0_to_usd:.2f})')
print(f'A) [Our front-run]       token1_out: {A_amount_token1_out} (${A_amount_token1_out * token1_to_usd:.2f})')
print(f'B) [3rd party large txn] token0_in:  {B_amount_token0_in}  (-${B_amount_token0_in * token0_to_usd:.2f})')
print(f'B) [3rd party large txn] token1_out: {B_amount_token1_out} (${B_amount_token1_out * token1_to_usd:.2f})')
print(f'C) [Our swap after]      token1_in:  {C_amount_token1_in}  (-${C_amount_token1_in * token1_to_usd:.2f})')
print(f'C) [Our swap after]      token0_out: {C_amount_token0_out} (${C_amount_token0_out * token0_to_usd:.2f})')

token0_gain = C_amount_token0_out - A_amount_token0_in
print(f'Net WELL tokens gain (discounting gas fees and potential slippage): {token0_gain} (${token0_gain * token0_to_usd:.2f})')

3rd party's proportion of amountInMax: 0.999754115344432

Result
A) [Our front-run]       token0_in:  1e+22  (-$127.46)
A) [Our front-run]       token1_out: 2.2392516135250618e+20 ($160.15)
B) [3rd party large txn] token0_in:  5.031589398686225e+23  (-$6413.26)
B) [3rd party large txn] token1_out: 10000000000000000000000 ($7152.08)
C) [Our swap after]      token1_in:  2.2392516135250618e+20  (-$160.15)
C) [Our swap after]      token0_out: 1.2568630960834612e+22 ($160.20)
Net WELL tokens gain (discounting gas fees and potential slippage): 2.5686309608346123e+21 ($32.74)


# Conclusions
Front-running large transactions can clearly be profitable, but I think there are too few such transactions for it to be viable and worth the development effort.

## What is a 'large transaction'?
A 'large transaction' needs to have a relatively high dollar notional amount (so our profit is sizeable) AND also change the rate significantly (otherwise we lose revenue in fees). For particularly large liquidity pools (e.g. xcDOT/wGLMR), the latter becomes less likely. In this sense, we may actually **prefer front-running in less liquid pools** (see case 2 in the appendix).

## What we need to front-run successfully
In order to pull it off, we **need to be able to sandwich transactions** (most importantly, place a transaction before one in the mempool). I am not sure if the best way is to set higher gas limits, use our collator to reorder transactions and somehow get that ordering finalized ('MEV'), offer tips (https://wiki.polkadot.network/docs/learn-transaction-fees), or something else.

A little bit of math above can yield a formula to compute the ideal amount of tokens to swap, in order to maximize profit in such a sandwich trade.

## Caveats/Cons
1. Front-running will be very apparent on the blockchain and may be frowned upon. It is likely that these third parties may change their trading behavior after seeing our trades.
2. Our profit is particularly sensitive to the 'limit prices' (amountInMax, amountOutMin) so tighter bounds on those will cut deeply.
3. This strategy cannot scale with our amount of capital. There is a limited amount of money that can be made on each front-run transaction, bounded by the limit price of the swap we are front-running.
4. StellaSwap has the lowest fees of any DEX on Moonbeam, at 0.25% (most others are at 0.3%). It will thus be even more difficult to extend front-running to other venues.

## **How profitable is front-running then?**
Note that StellaSwap trading volume varies drastically day-to-day (some days are 10x other days, according to https://analytics.stellaswap.com/) but was at a low level for the analyzed period. Front-running is obviously more profitable when there is higher trading volume.

Disclaimer: I have not done a rigorous analysis of the data to calculate potential PnL (this can be done but will take several full days of effort), but I have done several 'case studies' attached in the appendix.
In a ~15 day span between June 24 and July 8 (blocks 1,300,000 - 1,400,000), **of all swaps that changed the rate by 2.5% change or more** (see case study 4 for the choice of this threshold),
```
>$250: 1.03% of all swaps -> 318 swaps
>$500: 0.77% of all swaps -> 240 swaps
>$750: 0.62% of all swaps -> 193 swaps
>$1000: 0.48% of all swaps -> 148 swaps
>$1250: 0.32% of all swaps -> 99 swaps
>$1500: 0.26% of all swaps -> 80 swaps
>$1750: 0.19% of all swaps -> 60 swaps
>$2000: 0.15% of all swaps -> 48 swaps
```
Again based on case study 4, we very roughly estimate a $\$$5 profit on each front-run opportunity of notional value $\$$1000. 148 * $\$$5 = $\$$740; even after adding the larger notional value or rate delta swaps, we likely to remain at less than a $\$$1000 PnL over 15 days i.e. <$\$$66 PnL per day.

# Appendix: analyzing a few more large transactions

### Case 1: Another best case scenario - A $1000 notional swap that changes the exchange rate significantly

In [6]:
# Txn hash: 0xc1c03e52f6c3b2bc6a221b19b06743674abe733d818fa337e3ae5a15814089e0 (USDC -> WGLMR)
# swapExactTokensForETH

token0_to_usd = 1017588866449.328125 * 1e-18
token1_to_usd = 0.684373 * 1e-18

amountIn = 1000000000
amountOutMin = 1440413226868216113221
path = ['0x818ec0A7Fe18Ff94269904fCED6AE3DaE6d6dC0b', '0xAcc15dC74880C9944775448304B263D191c6077F']

r0 = 76072981111
r1 = 113108510548453041897472
print('Proportion of new rate to old rate:', ( (r0 + amountIn) / (r1 - amountOutMin) ) / (r0/ r1) )

# A) Front-run large transaction: swap token0 -> token1
A_amount_token0_in = 6e8
A_amount_token1_out = quote_with_fees(r0, r1, amount_in=A_amount_token0_in)
r0 += A_amount_token0_in
r1 -= A_amount_token1_out

# B) Large transaction occurs
B_amount_token0_in = amountIn
B_amount_token1_out = quote_with_fees(r0, r1, B_amount_token0_in)
assert(B_amount_token1_out >= amountOutMin) # otherwise their transaction will be rejected!
print('3rd party\'s proportion of amountOutMin to amountOut:', amountOutMin / B_amount_token1_out)
r0 += B_amount_token0_in
r1 -= B_amount_token1_out

# C) We swap in the reverse direction (token1 -> token0); for simplicity let's say we swap away all the token1's we acquired
C_amount_token1_in = A_amount_token1_out
C_amount_token0_out = quote_with_fees(r1, r0, C_amount_token1_in)

# Net gain:
print('\nResult')
print(f'A) [Our front-run]       token0_in:  {A_amount_token0_in}  (-${A_amount_token0_in * token0_to_usd:.2f})')
print(f'A) [Our front-run]       token1_out: {A_amount_token1_out} (${A_amount_token1_out * token1_to_usd:.2f})')
print(f'B) [3rd party large txn] token0_in:  {B_amount_token0_in}  (-${B_amount_token0_in * token0_to_usd:.2f})')
print(f'B) [3rd party large txn] token1_out: {B_amount_token1_out} (${B_amount_token1_out * token1_to_usd:.2f})')
print(f'C) [Our swap after]      token1_in:  {C_amount_token1_in}  (-${C_amount_token1_in * token1_to_usd:.2f})')
print(f'C) [Our swap after]      token0_out: {C_amount_token0_out} (${C_amount_token0_out * token0_to_usd:.2f})')

token0_gain = C_amount_token0_out - A_amount_token0_in
print(f'Net USDC tokens gain (discounting gas fees and potential slippage): {token0_gain} (${token0_gain * token0_to_usd:.2f})')

Proportion of new rate to old rate: 1.0262138909079448
3rd party's proportion of amountOutMin to amountOut: 0.999397653739576

Result
A) [Our front-run]       token0_in:  600000000.0  (-$610.55)
A) [Our front-run]       token1_out: 8.829286011215053e+20 ($604.25)
B) [3rd party large txn] token0_in:  1000000000  (-$1017.59)
B) [3rd party large txn] token1_out: 1.4412813773160613e+21 ($986.37)
C) [Our swap after]      token1_in:  8.829286011215053e+20  (-$604.25)
C) [Our swap after]      token0_out: 612620193.7075363 ($623.40)
Net USDC tokens gain (discounting gas fees and potential slippage): 12620193.70753634 ($12.84)


### Case 2: Lose money when front-running a $1000 notional swap that does *not* change the rate significantly

In [7]:
# Txn hash: 0xd4bb81de3f33b015fb03a8331eabe4ea9ec27c997da9dcf77dcf66cbce1939b6 (Reverse xcDOT -> WGLMR)
# swapExactTokensForETH

token0_to_usd = 0.682500 * 1e-18
token1_to_usd = 812521545.659391 * 1e-18

amountIn = 1350000000000
amountOutMin = 1595231890061043355982
path = ['0xFfFFfFff1FcaCBd218EDc0EbA20Fc2308C778080', '0xAcc15dC74880C9944775448304B263D191c6077F']

r0 = 3147793525006846568431616
r1 = 2641378280640074

print('Proportion of new rate to old rate:', ( (r1 + amountIn) / (r0 - amountOutMin) ) / (r1/r0))

# A) Front-run large transaction: swap token1 -> token0
A_amount_token1_in = 7e12
A_amount_token0_out = quote_with_fees(r1, r0, amount_in=A_amount_token1_in)
r0 -= A_amount_token0_out
r1 += A_amount_token1_in

# B) Large transaction occurs: token1 -> token0
B_amount_token1_in = amountIn
B_amount_token0_out = quote_with_fees(r1, r0, B_amount_token1_in)
assert(B_amount_token0_out >= amountOutMin) # otherwise their transaction will be rejected!
print('3rd party\'s proportion of amountOutMin to amountOut:', amountOutMin / B_amount_token0_out)
r0 -= B_amount_token0_out
r1 += B_amount_token1_in

# C) We swap in the reverse direction (token0 -> token1); for simplicity let's say we swap away all the token1's we acquired
C_amount_token0_in = A_amount_token0_out
C_amount_token1_out = quote_with_fees(r0, r1, C_amount_token0_in)

# Net gain:
print('\nResult')
print(f'A) [Our front-run]       token1_in:  {A_amount_token1_in}  (-${A_amount_token1_in * token1_to_usd:.2f})')
print(f'A) [Our front-run]       token0_out: {A_amount_token0_out} (${A_amount_token0_out * token0_to_usd:.2f})')
print(f'B) [3rd party large txn] token1_in:  {B_amount_token1_in}  (-${B_amount_token1_in * token1_to_usd:.2f})')
print(f'B) [3rd party large txn] token0_out: {B_amount_token0_out} (${B_amount_token0_out * token0_to_usd:.2f})')
print(f'C) [Our swap after]      token0_in:  {C_amount_token0_in}  (-${C_amount_token0_in * token0_to_usd:.2f})')
print(f'C) [Our swap after]      token1_out: {C_amount_token1_out} (${C_amount_token1_out * token1_to_usd:.2f})')

token1_gain = C_amount_token1_out - A_amount_token1_in
print(f'Net WGLMR tokens gain (discounting gas fees and potential slippage): {token1_gain} (${token1_gain * token1_to_usd:.2f})')
print('We actually lose money here because this liquidity pool is too large; xcDOT/WGLMR is the largest liquidity pool by far according to analytics.stellaswap.com/')

Proportion of new rate to old rate: 1.001018390706798
3rd party's proportion of amountOutMin to amountOut: 0.9998117713818307

Result
A) [Our front-run]       token1_in:  7000000000000.0  (-$5687.65)
A) [Our front-run]       token0_out: 8.299272685592391e+21 ($5664.25)
B) [3rd party large txn] token1_in:  1350000000000  (-$1096.90)
B) [3rd party large txn] token0_out: 1.5955322148850958e+21 ($1088.95)
C) [Our swap after]      token0_in:  8.299272685592391e+21  (-$5664.25)
C) [Our swap after]      token1_out: 6972220188000.216 ($5665.08)
Net WGLMR tokens gain (discounting gas fees and potential slippage): -27779811999.78418 ($-22.57)
We actually lose money here because this liquidity pool is too large; xcDOT/WGLMR is the largest liquidity pool by far according to analytics.stellaswap.com/


### Case 3: Break even on a $300 notional swap with a 0.75% exchange rate delta

In [8]:
# Txn hash: 0x44a5c0224d173b53f6a72f13e77992d3b69f498e7c9907c7acf838c9bc09679f (Reverse AVAX -> WGLMR)
# swapExactTokensForTokens

token0_to_usd = 16.697423 * 1e-18
token1_to_usd = 0.606330 * 1e-18

amountIn = 472148999999999967232
amountOutMin = 17055264230515683799
path = ['0xAcc15dC74880C9944775448304B263D191c6077F', '0x4792C1EcB969B036eb51330c63bD27899A13D84e']

r0 = 4521096583621023105024
r1 = 123259246444304522018816

print('Proportion of new rate to old rate:', ( (r1 + amountIn) / (r0 - amountOutMin) ) / (r1/r0) )

# A) Front-run large transaction: swap token1 -> token0
A_amount_token1_in = 5e20
A_amount_token0_out = quote_with_fees(r1, r0, amount_in=A_amount_token1_in)
r0 -= A_amount_token0_out
r1 += A_amount_token1_in

# B) Large transaction occurs: token1 -> token0
B_amount_token1_in = amountIn
B_amount_token0_out = quote_with_fees(r1, r0, B_amount_token1_in)
assert(B_amount_token0_out >= amountOutMin) # otherwise their transaction will be rejected!
print('3rd party\'s proportion of amountOutMin to amountOut:', amountOutMin / B_amount_token0_out)
r0 -= B_amount_token0_out
r1 += B_amount_token1_in

# C) We swap in the reverse direction (token0 -> token1); for simplicity let's say we swap away all the token1's we acquired
C_amount_token0_in = A_amount_token0_out
C_amount_token1_out = quote_with_fees(r0, r1, C_amount_token0_in)

# Net gain:
print('\nResult')
print(f'A) [Our front-run]       token1_in:  {A_amount_token1_in}  (-${A_amount_token1_in * token1_to_usd:.2f})')
print(f'A) [Our front-run]       token0_out: {A_amount_token0_out} (${A_amount_token0_out * token0_to_usd:.2f})')
print(f'B) [3rd party large txn] token1_in:  {B_amount_token1_in}  (-${B_amount_token1_in * token1_to_usd:.2f})')
print(f'B) [3rd party large txn] token0_out: {B_amount_token0_out} (${B_amount_token0_out * token0_to_usd:.2f})')
print(f'C) [Our swap after]      token0_in:  {C_amount_token0_in}  (-${C_amount_token0_in * token0_to_usd:.2f})')
print(f'C) [Our swap after]      token1_out: {C_amount_token1_out} (${C_amount_token1_out * token1_to_usd:.2f})')

token1_gain = C_amount_token1_out - A_amount_token1_in
print(f'Net WGLMR tokens gain (discounting gas fees and potential slippage): {token1_gain} (${token1_gain * token1_to_usd:.2f})')
print('We effectively break even here')

Proportion of new rate to old rate: 1.0076316991161764
3rd party's proportion of amountOutMin to amountOut: 0.9990878724928451

Result
A) [Our front-run]       token1_in:  5e+20  (-$303.17)
A) [Our front-run]       token0_out: 1.822021191249463e+19 ($304.23)
B) [3rd party large txn] token1_in:  472148999999999967232  (-$286.28)
B) [3rd party large txn] token0_out: 1.707083500869722e+19 ($285.04)
C) [Our swap after]      token0_in:  1.822021191249463e+19  (-$304.23)
C) [Our swap after]      token1_out: 5.013040568569609e+20 ($303.96)
Net WGLMR tokens gain (discounting gas fees and potential slippage): 1.304056856960893e+18 ($0.79)
We effectively break even here


### Case 4: Most realistic scenario - A \\$5 profit when front-running a $1000 notional swap with a 2.5% rate delta

In [9]:
# Txn hash: 0x8c35343b6920d3956ff69950df113fa625fafd7550c24cda28b79d42a89ee7a0 (axlATOM -> WGLMR)
# swapTokensForExactTokens
# This txn crosses two liquidity pools; we front-run the swap on just the first liquidity pool

amountOut = 1482914392357657903104
amountInMax = 129634932

token0_to_usd = 8042484633203.593750 * 1e-18
token1_to_usd = 0.706221 * 1e-18

r0 = 9865163938
r1 = 115295378813434539802624

print('Proportion of new rate to old rate:', ( (r0 + amountInMax) / (r1 - amountOut) ) / (r0/r1) )

# A) Front-run large transaction: swap token0 -> token1
A_amount_token0_in = 2.5e7
A_amount_token1_out = quote_with_fees(r0, r1, amount_in=A_amount_token0_in)
r0 += A_amount_token0_in
r1 -= A_amount_token1_out

# B) Large transaction occurs
B_amount_token1_out = amountOut
B_amount_token0_in = inverse_quote_with_fees(r0, r1, B_amount_token1_out)
assert(B_amount_token0_in <= amountInMax) # otherwise their transaction will be rejected!
print('3rd party\'s proportion of amountIn to amountInMax:', B_amount_token0_in / amountInMax)
r0 += B_amount_token0_in
r1 -= B_amount_token1_out

# C) We swap in the reverse direction (token1 -> token0); for simplicity let's say we swap away all the token1's we acquired
C_amount_token1_in = A_amount_token1_out
C_amount_token0_out = quote_with_fees(r1, r0, C_amount_token1_in)

# Net gain:
print('\nResult')
print(f'A) [Our front-run]       token0_in:  {A_amount_token0_in}  (-${A_amount_token0_in * token0_to_usd:.2f})')
print(f'A) [Our front-run]       token1_out: {A_amount_token1_out} (${A_amount_token1_out * token1_to_usd:.2f})')
print(f'B) [3rd party large txn] token0_in:  {B_amount_token0_in}  (-${B_amount_token0_in * token0_to_usd:.2f})')
print(f'B) [3rd party large txn] token1_out: {B_amount_token1_out} (${B_amount_token1_out * token1_to_usd:.2f})')
print(f'C) [Our swap after]      token1_in:  {C_amount_token1_in}  (-${C_amount_token1_in * token1_to_usd:.2f})')
print(f'C) [Our swap after]      token0_out: {C_amount_token0_out} (${C_amount_token0_out * token0_to_usd:.2f})')

token0_gain = C_amount_token0_out - A_amount_token0_in
print(f'Net axlATOM tokens gain (discounting gas fees and potential slippage): {token0_gain} (${token0_gain * token0_to_usd:.2f})')

Proportion of new rate to old rate: 1.0263413478490346
3rd party's proportion of amountIn to amountInMax: 0.9990921307531818

Result
A) [Our front-run]       token0_in:  25000000.0  (-$201.06)
A) [Our front-run]       token1_out: 2.9071274214771343e+20 ($205.31)
B) [3rd party large txn] token0_in:  129517240.43192384  (-$1041.64)
B) [3rd party large txn] token1_out: 1482914392357657903104 ($1047.27)
C) [Our swap after]      token1_in:  2.9071274214771343e+20  (-$205.31)
C) [Our swap after]      token0_out: 25529588.85103076 ($205.32)
Net axlATOM tokens gain (discounting gas fees and potential slippage): 529588.8510307595 ($4.26)
